In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("train.csv")
x = df[["Age", "BMI", "HbA1c Level", "Fasting Blood Glucose", "High Blood Pressure", "Smoking"]].to_numpy()
y = df["Diagnosis"].to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, x_train, y_train, return_train_score=True, n_jobs=-1)

print("훈련 세트:", np.mean(scores["train_score"]))
print("시험 세트:", np.mean(scores["test_score"]))

훈련 세트: 0.9968837224292685
시험 세트: 0.963962464024749


In [22]:
from sklearn.model_selection import GridSearchCV

params = {"n_estimators": range(5,50,10),
          "max_features": range(1,7),
          'max_depth': range(3,5)
          }


gs = GridSearchCV(estimator=rf, param_grid=params,scoring='accuracy' ,n_jobs=-1)

gs.fit(x_train, y_train)

c:\Users\USER\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1, oob_score=True,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': range(3, 5), 'max_features': range(1, 7),
                         'n_estimators': range(5, 50, 10)},
             scoring='accuracy')

In [21]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(x_train, y_train)
print(rf.oob_score_)

0.9624160820698499


In [24]:
print(gs.best_params_)

{'max_depth': 3, 'max_features': 3, 'n_estimators': 15}


In [25]:
rf = gs.best_estimator_
print(rf.score(x_train, y_train))
print(rf.score(x_test, y_test))

0.9682997661612733
0.9685449196650826


In [5]:
rf.fit(x_train, y_train)
print(rf.feature_importances_)

[0.12858371 0.09067411 0.409766   0.34957268 0.01215457 0.00924892]


- 그레이디언트 부스팅

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

gb = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=42)
scores = cross_validate(gb, x_train, y_train, return_train_score=True, n_jobs=-1)

print("훈련 세트:", np.mean(scores["train_score"]))
print("시험 세트:", np.mean(scores["test_score"]))

훈련 세트: 0.9694736743923439
시험 세트: 0.9684694980637311


In [7]:
from sklearn.model_selection import GridSearchCV

params = {"n_estimators": range(5,70,10),
          "max_features": range(6),
          'max_depth': range(3,5)
          }


gs = GridSearchCV(estimator=gb, param_grid=params,scoring='accuracy' ,n_jobs=-1)

gs.fit(x_train, y_train)

c:\Users\USER\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
70 fits failed out of a total of 420.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USER\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 420, in fit
    self._validate_params()
  File "c:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py", line 581, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\USER\anaconda3\lib\site-packages\sklearn

GridSearchCV(estimator=GradientBoostingClassifier(n_estimators=200,
                                                  random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': range(3, 5), 'max_features': range(0, 6),
                         'n_estimators': range(5, 70, 10)},
             scoring='accuracy')

In [8]:
gb = gs.best_estimator_
print(gb.score(x_train, y_train))
print(gb.score(x_test, y_test))

0.9684694878177567
0.9688277890925548


In [9]:
print(gs.best_params_)

{'max_depth': 4, 'max_features': 3, 'n_estimators': 55}


In [10]:
gb.fit(x_train, y_train)
print(gb.feature_importances_)

[4.76196696e-02 1.63779082e-02 5.30858843e-01 3.96985731e-01
 7.98076407e-03 1.77084084e-04]


In [11]:
df2 = pd.read_csv('test.csv')

y_predicted = df2[["Age", "BMI", "HbA1c Level", "Fasting Blood Glucose", "High Blood Pressure", "Smoking"]].to_numpy()

In [12]:
y_predict = gb.predict(y_predicted)

df2["Diagnosis"] = y_predict
df2

,Age,BMI,High Blood Pressure,Fasting Blood Glucose,HbA1c Level,Smoking,Diagnosis
0,51,20,0,159,6.2,1,0
1,14,20,0,158,6.1,0,0
2,38,27,0,200,5.8,0,0
3,16,17,0,126,3.5,1,0
4,80,31,1,100,4.8,0,0
...,...,...,...,...,...,...,...
17671,8,23,0,126,5.7,0,0
17672,43,27,0,80,5.8,0,0
17673,11,20,0,160,5.8,0,0
17674,24,27,0,90,5.8,0,0


In [13]:
import pandas as pd
import numpy as np

f = pd.read_csv("submission.csv")

f.dropna(axis=1, inplace=True)

# 아래 "np.arange(0, df.shape[0], 1)" 부분을, 솜솜이가 만든 모델이 예측한 값으로 대체!
y_pred = y_predict 

f["Diagnosis"] = y_pred
f.to_csv("new_submission.csv", index=False)